In [1]:
# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
# # Clone github repo for this project
# !git clone https://github.com/RyanJenkinson/nlp-jcag.git

In [5]:
!pip install bert-tensorflow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re
import sys

from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

import bert
from bert import run_classifier

OUTPUT_BASE_DIR = "/content/gdrive/My Drive/Bert/"

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

W0316 16:22:18.131930 140374650193792 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [6]:
# Import data
df = pd.read_csv(OUTPUT_BASE_DIR+'jokes_dataset_CLEAN.csv')
df = df.drop(columns="Unnamed: 0")
df.head()

,body,id,score,title
0,"Now I have to say ""Leroy can you please paint ...",5tz52q,1,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the ov...,5tz4dd,0,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,5tz319,0,I recently went to America....
3,He got caught trying to sell the two books to ...,5tz1pc,0,You hear about the University book store worke...
4,Because the p is silent.,5tz1o1,0,Why is it unknown on how pterodactyls urinate ...


In [0]:
def preprocess(words, to_lowercase=True, remove_punctuation=True, remove_digits=False, 
               remove_odd_chars=True, remove_stopwords=False, stem=False, spell_check=False,
               lemmatize=False):
    
    if to_lowercase:
        words = [w.lower() for w in words]
    
    if remove_punctuation:
        words = [w for w in words if not (re.match(r'^\W+$', w) != None)]
    
    if remove_digits:
        words = [w for w in words if not w.replace('.','',1).isdigit()]

    if remove_odd_chars:
        words = [re.sub(r'[^a-zA-Z0-9_]','', w) for w in words]
    
    if remove_stopwords:
        sw = set(nltk.corpus.stopwords.words("english"))
        words = [w for w in words if not w in sw]

    if spell_check:
        words = [str(TextBlob(w).correct()) for w in words]
     
    if stem:
        porter = nltk.PorterStemmer()
        words = [porter.stem(w) for w in words]

    if lemmatize:
        wordnet = nltk.WordNetLemmatizer()
        words = [wordnet.lemmatize(w) for w in words]
        
    return words

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        return {"eval_accuracy": accuracy}

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [10]:
# Define classes for classification #df.score[i] > 3 and
df['class'] = [0 if df.score[i] <= 2 \
               else 1 if (df.score[i] > 2 and df.score[i] <=50) \
               else 2 for i in range(df.shape[0])]
print(df['class'].value_counts())

# Select an even sample of classes
even_sample = df[df['class']==0].sample(frac=1).reset_index(drop=True)[:16759]
even_sample = even_sample.append(df[df['class']==1].sample(frac=1).reset_index(drop=True)[:16759])
even_sample = even_sample.append(df[df['class']==2][:16759])
even_sample = even_sample.sample(frac=1).reset_index(drop=True)
print(even_sample['class'].value_counts())

def encode_jokes(even_sample):
  # Concatonate set-up and punchline
  docs = [str(even_sample.title[i]) + ' ' + str(even_sample.body[i]) for i in range(even_sample.shape[0])]
  return docs#docs_padded

even_sample['joke'] = encode_jokes(even_sample)

0    75824
1    53258
2    16759
Name: class, dtype: int64
2    16759
1    16759
0    16759
Name: class, dtype: int64


In [0]:
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

In [12]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 70
label_list=[0,1,2]

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Run 10 times
accur_10 = []
for run_num in range(8,9):
  OUTPUT_DIR = OUTPUT_BASE_DIR+str(run_num)+"/"
  df_train, df_test = train_test_split(even_sample, test_size=0.2, random_state=0)
  df_train = df_train.reset_index(drop=True)
  df_test = df_test.reset_index(drop=True)


  # Use the InputExample class from BERT's run_classifier code to create examples from the data
  train_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                     text_a = x['joke'], 
                                                                     text_b = None, 
                                                                     label = x['class']), axis = 1)

  test_InputExamples = df_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                     text_a = x['joke'], 
                                                                     text_b = None, 
                                                                     label = x['class']), axis = 1)

  tokenizer = create_tokenizer_from_hub_module()

  # Convert our train and test features to InputFeatures that BERT understands.
  train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  
  # Compute # train and warmup steps from batch size
  num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

  # Specify outpit directory and number of checkpoint steps to save
  run_config = tf.estimator.RunConfig(
      model_dir=OUTPUT_DIR,
      save_summary_steps=SAVE_SUMMARY_STEPS,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

  model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps)

  estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE})

  # Create an input function for training. drop_remainder = True for using TPUs.
  train_input_fn = bert.run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=False)

  print(f'Beginning Training!')
  current_time = datetime.now()
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print("Training took time ", datetime.now() - current_time)

  test_input_fn = run_classifier.input_fn_builder(
      features=test_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=False)

  accur_10.append(estimator.evaluate(input_fn=test_input_fn, steps=None))
  
  with open(OUTPUT_BASE_DIR + "accur_"+str(run_num)+".pickle", 'wb') as handle:
    pickle.dump(accur_10, handle)
  files.download(OUTPUT_BASE_DIR + "accur_"+str(run_num)+".pickle")
    

Instructions for updating:
Colocations handled automatically by placer.


W0316 16:22:27.987132 140374650193792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 16:22:30.784500 140374650193792 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 40221


...

INFO:tensorflow:Saving dict for global step 3770: eval_accuracy = 0.46071997, global_step = 3770, loss = 1.4520912


I0316 17:27:40.111842 140374650193792 estimator.py:1979] Saving dict for global step 3770: eval_accuracy = 0.46071997, global_step = 3770, loss = 1.4520912


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3770: /content/gdrive/My Drive/Bert/8/model.ckpt-3770


I0316 17:27:43.366577 140374650193792 estimator.py:2039] Saving 'checkpoint_path' summary for global step 3770: /content/gdrive/My Drive/Bert/8/model.ckpt-3770


KeyboardInterrupt: ignored

In [0]:
# bert_predict = pd.DataFrame(vardlkdsjf)
# cm = metrics.confusion_matrix(df_test['class'], bert_predict['labels'])
# plt.figure(figsize = (5,2))
# sns.heatmap(cm, annot=True,fmt='g')
# plt.ylabel('True label')
# plt.xlabel('BERT Predicted label')
# plt.savefig(OUTPUT_DIR+'BERTModelConfusionMatrix.png', bbox_inches='tight')
# plt.show()